In [1]:
import vgg16
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()

In [3]:
print(len(text))
print(text[:100])

600901
preface


supposing that truth is a woman--what then? is there not ground
for suspecting that all ph


In [4]:
!tail -n25 {path}

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [5]:
chars = sorted(list(set(text)))
chars.insert(0, '\0') # Make the index starting from 1
print(chars)
print(len(chars))
''.join(chars)

['\x00', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x86', '\xa4', '\xa6', '\xa9', '\xab', '\xc3']
60


'\x00\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz\x86\xa4\xa6\xa9\xab\xc3'

In [6]:
char_to_index = {c:i for i, c in enumerate(chars)}
index_to_char = {i:c for i, c in enumerate(chars)}

In [7]:
indexes = [char_to_index[c] for c in text]

In [8]:
''.join([index_to_char[i] for i in indexes[:50]])

'preface\n\n\nsupposing that truth is a woman--what th'

In [9]:
max_len = 40
sentences = []
next_chars = []
for i in range(len(indexes)- max_len):
    sentences.append(indexes[i:i+max_len])
    next_chars.append(indexes[i+1:i+max_len+1])

In [10]:
print(len(sentences))
print(len(next_chars))
#print(text[:50])
print("%s\n%s"%(sentences[-1], next_chars[-1]))

600861
600861
[41, 2, 50, 42, 45, 39, 31, 2, 28, 46, 2, 47, 35, 32, 2, 36, 41, 31, 36, 30, 28, 47, 36, 42, 41, 46, 2, 42, 33, 2, 46, 36, 41, 33, 48, 39, 41, 32, 46, 46]
[2, 50, 42, 45, 39, 31, 2, 28, 46, 2, 47, 35, 32, 2, 36, 41, 31, 36, 30, 28, 47, 36, 42, 41, 46, 2, 42, 33, 2, 46, 36, 41, 33, 48, 39, 41, 32, 46, 46, 10]


In [11]:
#[np.array(s) for s in sentences]
#m = np.concatenate([np.array([s for s in ss]) for ss in sentences])
sentences1 = np.row_stack([[s for s in ss] for ss in sentences])
next_chars1 = np.row_stack([[s for s in ss] for ss in next_chars])
print(sentences1.shape)
print(next_chars1.shape)

(600861, 40)
(600861, 40)


In [12]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [13]:
print(sentences.shape)
print(next_chars.shape)

(600859, 40)
(600859, 40)


In [14]:
np.array(sentences[0])

array([43, 45, 32, 33, 28, 30, 32,  1,  1,  1, 46, 48, 43, 43, 42, 46, 36, 41, 34,  2, 47, 35, 28,
       47,  2, 47, 45, 48, 47, 35,  2, 36, 46,  2, 28,  2, 50, 42, 40, 28])

In [16]:
print(next_chars.shape)

(600859, 40)


In [195]:
sentences1[0]

array(['p', 'r', 'e', 'f', 'a', 'c', 'e', '\n', '\n', '\n', 's', 'u', 'p', 'p', 'o', 's', 'i', 'n',
       'g', ' ', 't', 'h', 'a', 't', ' ', 't', 'r', 'u', 't', 'h', ' ', 'i', 's', ' ', 'a', ' ',
       'w', 'o', 'm', 'a'],
      dtype='|S1')

In [17]:
n_fac = 24
vocab_size = len(chars)

In [23]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_to_index[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [19]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=max_len),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, recurrent_dropout=0.2, implementation=2, dropout=0.2, input_shape=(None, 24), return_sequences=True)`
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, recurrent_dropout=0.2, implementation=2, dropout=0.2, return_sequences=True)`


In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [21]:
model.fit(sentences1, np.expand_dims(next_chars1,-1), batch_size=64, nb_epoch=1)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
600861/600861 [==============================] - 2618s 4ms/step - loss: 1.4950


In [24]:
print_example()

ethics is a basic foundation of all that is seldom fundamental tendency than complex to the free soul, in the metaphysicians, and
here and should aver to exist and as. finally, granting to
which i had not shall be the most delicate and
melanchology of will.--"inspires his "one widen" feels him (are something highly
believed in everything problem of man. a
wi
